### Интерактивное приложение в формате Notebook для построение моделей машинного обучения для прогнозирования параметров:
#### "Модуль упругости при растяжении"
#### "Прочность при растяжении"

#### "Соотношение матрица-наполнитель" с использованием нейросети.

In [ ]:
# Подгружаем необходимые библиотеки
import pickle
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Загрузка моделей из файлов
with open('pipeline_KNeighborsRegressor_model_1.pkl', 'rb') as file:
    KNeighborsRegressor_model_1 = pickle.load(file)

with open('pipeline_AdaBoostRegressor_model_2.pkl', 'rb') as file:
    AdaBoostRegressor_model_2 = pickle.load(file)

NN_Sequental_model_3 = keras.models.load_model('NN_Sequental_model_3.keras')

# Подготовка входных данных (10 параметров в виде списка)
input_data = pd.DataFrame({
    'Плотность': [float(input
                        (f"Введите значение Плотности (в интервале 1801-2150 кг/м3): "))],
    'Модуль упругости': [float(input
                               (f"Введите значение Модуля упругости (в интервале 2.43-1588 ГПа): "))],
    'Количество отвердителя': [float(input
                                     (f"Введите значение Количества отвердителя (в интервале 41.8-181.8 м.%): "))], 
    'Содержание эпоксидных групп': [float(input
                                          (f"Введите значение Содержания эпоксидных групп (в интервале 15.88-28.62 %_2): "))],
    'Температура вспышки': [float(input
                                  (f"Введите значение Температуры вспышки (в интервале 186-385 С_2): "))],
    'Поверхностная плотность': [float(input
                                      (f"Введите значение Поверхностной плотности (в интервале 0.6-1238 г/м2): "))],
    'Потребление смолы': [float(input
                                (f"Введите значение Потребления смолы (в интервале 72-359 г/м2): "))],
    'Угол нашивки': [float(input
                           (f"Введите значение Угла нашивки (0 или 90): "))],
    'Шаг нашивки': [float(input
                          (f"Введите значение Шага нашивки (в интервале 0.14-13.48): "))],
    'Плотность нашивки': [float(input
                                (f"Введите значение Плотности нашивки (в интервале 39.33-80): "))]
})

input_data['Угол нашивки'] = input_data['Угол нашивки'].astype('category')

# Выполнение прогноза Прочности при растяжении
prediction_param1 = KNeighborsRegressor_model_1.predict(input_data)

# Выполнение прогноза Модуля упругости при растяжении
prediction_param2 = AdaBoostRegressor_model_2.predict(input_data)

# Применение обученной нейросети
# Определение числовых и категориальных данных
numeric_features = input_data.select_dtypes(include=['float64']).columns
categorical_features = input_data.select_dtypes(include=['category']).columns

# Создание преобразователей для числовых и категориальных данных
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(categories=[[0, 90]], sparse_output=False))
])

# Объединение преобразований в ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Применение предобработки
NN_input_data = preprocessor.fit_transform(input_data)

prediction_param3 = NN_Sequental_model_3.predict(NN_input_data)

# Вывод результата 
print(f"Результат прогноза Прочности при растяжении: {prediction_param1[0]}, МПа")
print("=" * 70)
print(f"Результат прогноза Модуля упругости при растяжении: {prediction_param2[0]}, ГПа")
print("=" * 70)
print(f"Результат прогноза Соотношение матрица-наполнитель: {prediction_param3[0]} ")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Результат прогноза Прочности при растяжении: [2187.35792238], МПа
Результат прогноза Модуля упругости при растяжении: 73.19572857508987, ГПа
Результат прогноза Соотношение матрица-наполнитель: [3.0880105] 


In [13]:
input_data

,Плотность,Модуль упругости,Количество отвердителя,Содержание эпоксидных групп,Температура вспышки,Поверхностная плотность,Потребление смолы,Угол нашивки,Шаг нашивки,Плотность нашивки
0,2100.0,1400.0,170.0,25.0,320.0,1100.0,280.0,0.0,10.0,60.0
